In [2]:
# run this cell to check your Python version is OK for this notebook!
import sys
def check_python_version_above_3_6():
    major = sys.version_info.major
    minor = sys.version_info.minor
    if major < 3 or minor < 6:
        print('ERROR you need to run this notebook with Python 3.6 or above (as f-strings used)')
        print('ERROR current Python version is {}.{}'.format(major, minor))        
        print('ERROR Please see:\n',
              '      https://canvas.anglia.ac.uk/courses/15139/pages/azure-notebooks-switching-kernel\n'
              '      for information on switching kernel on Azure Notebooks')
    else:
        print('Python version {}.{} you are good to go'.format(major, minor))
check_python_version_above_3_6()

Python version 3.8 you are good to go


# BLAST method for database searching
This practical uses the simple BLAST algorithm 'MyBlast' from Chapter 7 of Rocha and Ferreira (2018) Bioinformatics Algorithms. 

It is not supposed to be a fully-fledged program but aims only to show the initial steps in the database search method. 

The first step is to create a database. This will be a text file containing a set of strings representing the sequencees to be searched. Each sequence is short enough in the test data to be given as a single line in the file. 

The authors provide a test data file callled `seqBlast.txt` that is supplied with this notebook for available from https://github.com/ARU-Bioinf-CMA-2020/tw6 

The function read_database opens that file and creates the database.

In [3]:
# run this cell to define function read_database
def read_database (filename):    
    """"
    reads the sequences to search from a text file and returns as a list of strings.
    """
    file = open (filename)
    db = []
    for line in file:
        db.append(line.rstrip())
    file.close()
    return db

In [4]:
# run this cell to use read_database to read in entries from seqBlast.txt 
# and check that 5 entries read.
db = read_database("seqBlast.txt")
assert len(db) == 5

The next step is to pre-process the query sequence. The authors call this producing a 'map' of all word of a particular size (an adjustable parameter here - although remember that larger words are more demanding). A table of these can be a Python dictionary - as that naturally reproduces the hashing method described in the lecture. 

In [5]:
# run this cell to define the build_map function
def build_map(query, w):
    """
    preprocesses the query to store the positions of words
    """
    res = {}
    for i in range(len(query)-w+1):
        subseq = query[i:i+w]
        if subseq in res:
            res[subseq].append(i)
        else:
            res[subseq] = [i]
    return res

In the authors' simplified version of the algorithm they do not use a substitution table but instead a simple scoring function similar to the one used for the Smith-Waterman example. The function gives a positive score only to perfect hits. As explained in the lecture this has to be the default for many protein sequence words in BLOSUM62 as well. 

The next function get_hits will scan a sequence and find all matches of the words from the query map. These are considered 'hits'.

In [6]:
# run this cell to define get_hits
def get_hits (seq, map_, w):
    """
    scans the sequence for word hit in map
    returns tuples: 
    index of match in query with the
    index of match in sequence
    """    
    res = []  # list of tuples
    for i in range(len(seq)-w+1):
        subseq = seq[i:i+w]
        if subseq in map_:
            l = map_[subseq]
            for ind in l:
                res.append( (ind,i) )
    return res

The next step is to extend the hits found by the previous function. 

Here the the hit is extended while the new aligned positions score greated than or equal to half of the new positions.

No gaps are used. But a check needs to be made that the extension has not reached the end of either the query or the sequence. 

In [11]:
def extends_hit(seq, hit, query, w):
    """
    the hit positions are extended
    based on the sequence and query matches
    on either side
    
    returns a tuple:
    (the starting index of the alignment on the query,
     the starting index of the alignment on the sequences,
     the size of the alignment,
     and the score - that is the number of matching characters).
    """    
    stq, sts = hit[0], hit[1]
    ## extend hit forward
    matfw = 0       
    k=0
    bestk = 0
    while 2*matfw >= k and stq+w+k < len(query) and sts+w+k < len(seq):
        if query[stq+w+k] == seq[sts+w+k]: 
            matfw+=1
            bestk = k+1
        k += 1
    size = w + bestk
    ## extend hit backwards
    k = 0
    matbw = 0   
    bestk = 0
    while 2*matbw >= k and stq > k and sts > k:
        if query[stq-k-1] == seq[sts-k-1]: 
            matbw+=1
            bestk = k+1
        k+=1       
    size += bestk
    
    return (stq-bestk, sts-bestk, size, w+matfw+matbw)


The output for the extended hit is an alignment starting positions in the query and in the sequence, its size and finally a count of identity matches added to the word length. The words from hits are all fully identical by definition in this implementation. The identity count is a measure of the final score for the extended alignment.

The extend_hit function is applied to all the hits in turn by the following function hit_best_score. It then returns the top scoring one. 

In [12]:
def hit_best_score(seq, query, m, w):
    """
    the hit positions are extended
    based on the sequence and query matches
    on either side
    
    returns the best alignment the query an a given sequence
    as a tuple:
    (the starting index of the alignment on the query,
     the starting index of the alignment on the sequence,
     the size of the alignment,
     and the score - that is the number of matching characters).

    """   
    hits = get_hits(seq, m, w)
    bestScore = -1.0
    best = ()
    for h in hits:
        ext = extends_hit(seq, h, query, w)
        score = ext[3]
        if score > bestScore or (score== bestScore and ext[2] < best[2]):
            bestScore = score
            best = ext
    return best

The final step is to apply the previous functions to compare the query with all the sequences in the database. The best overall alignment is found for sequences with hits. The result is a tuple similar to the ones above. 

In [13]:
def best_alignment(db, query, w):
    """
    compare the query with all the sequences
    in the database 
    all significant scores >=0 are returned
    
    returns the best alignment the query an a given sequence
    as a tuple:
    (the starting index of the alignment on the query,
     the starting index of the alignment on the sequence,
     the size of the alignment,
     the score - that is the number of matching characters,
     The index of the sequence with the best alignment).
    """    
    m = build_map(query, w)
    bestScore = -1.0
    res = (0,0,0,0,0)
    for k in range(0,len(db)):
        bestSeq = hit_best_score(db[k], query, m, w)
        if bestSeq != ():
            score = bestSeq[3]  
            if score > bestScore or (score== bestScore and bestSeq[2] < res[2]):
                bestScore = score
                res = bestSeq[0], bestSeq[1], bestSeq[2], bestSeq[3], k
    if bestScore < 0:
        return ()
    else: 
        return res

Applying it to a database from the test data file.  The standard DNA word length of 11 is used:

In [40]:
query = "ggggcgacgacggcgacgaatgatg"
result = best_alignment(db, query, 11)
query_start, best_sequence_start, size, score, best_sequence_index = result # unpack result
print(f'alignment score {score} that is the number of matching characters')
print(f'alignment size {size} bases')
print(f'starting index of the alignment on the query {query_start}')
print(f'the best sequence is {best_sequence_index}')
print(f'starting index of the alignment on the best sequence {best_sequence_start}')

alignment score 20 that is the number of matching characters
alignment size 21 bases
starting index of the alignment on the query 4
the best sequence is 4
starting index of the alignment on the best sequence 0


In [41]:
# now your job is to print out the alignment between the query and the best sequence that has been identified
# this should look like:
# cgacgacggcgacgaatgatg
# |||||||| ||||||||||||
# cgacgacgacgacgaatgatg
# the vertical lines can be produced using the highlight_line function:

In [45]:
def highlight_line(first_seq, second_seq):
    """ 
    for the two sequences returns a line where matching letters are 
    highlighted with | except if the letter are a gap
    """
    joins = ['|' if a == b and a != '-' else ' ' for a, b in zip(first_seq, second_seq)]
    return ''.join(joins)
assert highlight_line('A', 'A') == '|'
assert highlight_line('AAAA', 'AAAA') == '||||'
assert highlight_line('AAAA', 'AGGA') == '|  |'
assert highlight_line('AA-AA', 'AA-AA') == '|| ||'

In [46]:
# Model answer remove from publication
best_sequence = db[best_sequence_index]
align_best = best_sequence[best_sequence_start:best_sequence_start+size]
align_query  = query[query_start:query_start+size]
print(align_query)
print(highlight_line(align_query, align_best))
print(align_best)

cgacgacggcgacgaatgatg
|||||||| ||||||||||||
cgacgacgacgacgaatgatg


# optional advanced exercise

Try out  a longer query - does your code for printing the alignment work for this.

In [52]:
query = "gacgcctcgcgctcgcgcgctgaggcaaaaaaaaaaaaaaaaaaaatcggatagctagctgagcgctcgatagcgcgttcgctgcatcgcgtatagcgctgaagctcccggcgagctgtctgtaaatcggatctcatctcgctctatcct"
r = best_alignment(db, query, 11)
result = best_alignment(db, query, 11)
query_start, best_sequence_start, size, score, best_sequence_index = result # unpack result
print(f'alignment score {score} that is the number of matching characters')
print(f'alignment size {size} bases')
print(f'starting index of the alignment on the query {query_start}')
print(f'the best sequence is {best_sequence_index}')
print(f'starting index of the alignment on the best sequence {best_sequence_start}')

alignment score 108 that is the number of matching characters
alignment size 149 bases
starting index of the alignment on the query 1
the best sequence is 3
starting index of the alignment on the best sequence 38


In [53]:
#